In [1]:
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import os, sys
sys.path.append('/home/ibecker/Dropbox/MyPythonClasses/FinalClass/')

import Classifier.Model_53.Multiband_53 as multiband
import BaseClass.plot_v2 as plot

In [2]:
# To reset cuda
# sudo rmmod nvidia_uvm
# sudo modprobe nvidia_uvm
# To see if the system regognises the GPU
device = 0
devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(devices[device], 'GPU')
tf.config.experimental.set_memory_growth(device=devices[device], enable=True)


2023-05-02 17:13:23.098023: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.098555: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.147434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.148134: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.148950: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from S

Read the TF-records of the test sets. Compute the mean magnitudes and final classification using the Functional API.

In [3]:
survey = 'Gaia'
label_order = ['CEP', 'T2CEP', 'MIRA_SR','RRAB', 'RRC', 'DSCT_SXPHE']

path = os.path.join('../03_Train_Models/', survey, 'Results/Fold_*',)
folds = glob(path)
folds.sort()
folds

['../03_Train_Models/Gaia/Results/Fold_1',
 '../03_Train_Models/Gaia/Results/Fold_2',
 '../03_Train_Models/Gaia/Results/Fold_3',
 '../03_Train_Models/Gaia/Results/Fold_4',
 '../03_Train_Models/Gaia/Results/Fold_5',
 '../03_Train_Models/Gaia/Results/Fold_6',
 '../03_Train_Models/Gaia/Results/Fold_7']

In [4]:
folder_name = 'Results'

In [5]:
# Containers to store the results
reports_folds = []
cm_folds = []
for fold in folds:
    tf.keras.backend.clear_session()
    # Set the fold path
    base_dir = fold+'/'
    
    # Get the last run
    path_runs_folder = os.path.join(base_dir, 'Models', '*')
    path_runs = glob(path_runs_folder)
    path_runs.sort()
    path_run = path_runs[-1]    
    
    path_preprocess = fold.replace('/03_Train_Models/', '/02_Create_Records/').replace('/'+folder_name+'/', '/Folds/')
    test_files = os.path.join(path_preprocess, 'test/*.tfrecord')
    path_preprocess = os.path.join(path_preprocess, 'metadata_preprocess.json')    

    
    new = multiband.Network()    
    settings_path = os.path.join(path_run, 'all_settings.json')

    fold_name = fold.split('/')[-1]
    write_path = os.path.join('./'+folder_name, fold_name+'_Results.dat')
    
    # Get weights path
    run = path_run.split('/')[-1]
    weights_path = os.path.join(fold, 'Models', run)
    new.run_test(settings_path, test_files, weights_path, df_paths = None);
    
#     result_path = os.path.join('./'+folder_name, fold.split('/')[-1]+'_Results.dat')
    
#     # From the results file, read the data and compute the classification scores
#     report_fold = plot.compute_classification_report(result_path)
#     reports_folds.append(report_fold)
#     # From the results file, read the data and compute the confussion matrix and the respective  labels 
#     cm_fold = plot.compute_confussion_matrices(result_path, labels=label_order)
#     cm_folds.append(cm_fold)
    break

../02_Create_Records/Gaia/Folds/Fold_1/test/*.tfrecord


2023-05-02 17:13:23.658398: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 17:13:23.660167: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.660946: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 17:13:23.661537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

              precision    recall  f1-score   support

         CEP       0.71      0.77      0.74      1166
  DSCT_SXPHE       0.78      0.58      0.67       680
     MIRA_SR       1.00      1.00      1.00      8000
        RRAB       0.86      0.91      0.89      8000
         RRC       0.79      0.71      0.75      3451
       T2CEP       0.64      0.52      0.57       236

    accuracy                           0.89     21533
   macro avg       0.80      0.75      0.77     21533
weighted avg       0.89      0.89      0.89     21533



In [6]:
for batch in new.dataset_test:
    break

Get the mean magnitude output

In [10]:
layers = new.models[0].layers
layer_name = layers[-2].name
intermediate_layer_model = tf.keras.Model(inputs=new.models[0].input,
                                 outputs=new.models[0].get_layer(layer_name).output)
mean_mags = intermediate_layer_model(batch[0])

Print the first example, as a sample. Not that the padding makes the entire array full.

In [32]:
mean_mags.numpy()[0]

array([18.545557, 18.530783, 18.514545, 18.491264, 18.357082, 18.34712 ,
       18.324821, 18.349573, 18.287626, 18.26964 , 18.292418, 18.31194 ,
       18.32886 , 18.343664, 18.356728, 18.36834 , 18.378729, 18.388079,
       18.39654 , 18.40423 , 18.411253, 18.41769 , 18.423613, 18.42908 ,
       18.43414 , 18.438839, 18.443214, 18.447298, 18.451118, 18.4547  ,
       18.458065, 18.461231, 18.464218, 18.467037, 18.469706, 18.472235,
       18.47463 , 18.47691 , 18.479076, 18.48114 , 18.483107, 18.484985,
       18.48678 , 18.488497, 18.490139, 18.491714, 18.493225, 18.494675,
       18.496069, 18.497408, 18.498697, 18.499939, 18.501135, 18.502289,
       18.503403, 18.504477, 18.505516, 18.506521, 18.50749 , 18.50843 ,
       18.509342, 18.510221, 18.511074, 18.511902, 18.512707, 18.513485,
       18.514244, 18.51498 , 18.515694, 18.51639 , 18.517065, 18.517723,
       18.518364, 18.518988, 18.519594, 18.520187, 18.520763, 18.521328,
       18.521875, 18.522411, 18.522934, 18.523443, 

Do the same operations as in the model, to get exactly what we want.

In [33]:
N = batch[0]['N_0'][0].numpy()
m0 = batch[0]['M0_0'][0].numpy()
data = batch[0]['input_LC_0'][0].numpy()[:N]
data

array([[ 1.4008764 ,  1.830699  ,  0.49304435, -0.72241086],
       [ 1.830699  ,  1.4313586 , -0.72241086,  0.2581554 ],
       [ 1.4313586 ,  1.8500041 ,  0.2581554 , -0.03686545],
       [ 1.8500041 ,  1.4696567 , -0.03686545, -0.07472663],
       [ 1.4696567 ,  1.7681835 , -0.07472663, -0.82287806],
       [ 1.7681835 ,  1.6156642 , -0.82287806,  0.72539264],
       [ 1.6156642 ,  1.2827781 ,  0.72539264, -0.13093238],
       [ 1.2827781 ,  1.8092461 , -0.13093238,  0.4258857 ],
       [ 1.8092461 ,  1.4851803 ,  0.4258857 , -0.9041752 ],
       [ 1.4851803 ,  1.576728  , -0.9041752 ,  0.40365925],
       [ 1.576728  ,  1.4341837 ,  0.40365925,  0.49392572]],
      dtype=float32)

In [39]:
tri_lower = np.tril(np.ones([N+1,N+1]),0)

# Third column and first delta of the third column
data1 = data[:,-1]
data0 = data[:1, 2]

data_ext = np.concatenate([data0, data1])

mags = np.matmul(tri_lower, data_ext)+m0

sum_mags = np.matmul(tri_lower, data_ext)+m0

sum_mags = sum_mags[1:]

N_obs = np.arange(3,N+3)

mean_mags_c = tf.divide(sum_mags, N_obs)

mean_mags[0]

<tf.Tensor: shape=(108,), dtype=float32, numpy=
array([18.545557, 18.530783, 18.514545, 18.491264, 18.357082, 18.34712 ,
       18.324821, 18.349573, 18.287626, 18.26964 , 18.292418, 18.31194 ,
       18.32886 , 18.343664, 18.356728, 18.36834 , 18.378729, 18.388079,
       18.39654 , 18.40423 , 18.411253, 18.41769 , 18.423613, 18.42908 ,
       18.43414 , 18.438839, 18.443214, 18.447298, 18.451118, 18.4547  ,
       18.458065, 18.461231, 18.464218, 18.467037, 18.469706, 18.472235,
       18.47463 , 18.47691 , 18.479076, 18.48114 , 18.483107, 18.484985,
       18.48678 , 18.488497, 18.490139, 18.491714, 18.493225, 18.494675,
       18.496069, 18.497408, 18.498697, 18.499939, 18.501135, 18.502289,
       18.503403, 18.504477, 18.505516, 18.506521, 18.50749 , 18.50843 ,
       18.509342, 18.510221, 18.511074, 18.511902, 18.512707, 18.513485,
       18.514244, 18.51498 , 18.515694, 18.51639 , 18.517065, 18.517723,
       18.518364, 18.518988, 18.519594, 18.520187, 18.520763, 18.521328,
   

Read the original file, in order to find the exact value

In [64]:
df_lc = pd.read_csv('/home/Data/Databases/GAIA/Consolidate_Gaia/data_3/LCs/BP_RP_v3/'+batch[0]['ID'][0].numpy().decode()+'.dat')

b = df_lc.band==1
mags_file = df_lc[b].mag.values


for i in range(3,mags_file.shape[0]+1):
    print(i, np.round(mags_file[:i].sum()/(i), 6))
print('final mean mag ', np.round(mags_file.sum()/mags_file.shape[0], 6))

3 18.545558
4 18.530783
5 18.514544
6 18.491264
7 18.357081
8 18.347118
9 18.324821
10 18.349572
11 18.287625
12 18.269641
13 18.292418
final mean mag  18.292418


In [67]:
mean_mag_tf

array([18.545557, 18.530783, 18.514545, 18.491264, 18.357082, 18.34712 ,
       18.324821, 18.349573, 18.287626, 18.26964 , 18.292418, 18.31194 ,
       18.32886 , 18.343664, 18.356728], dtype=float32)

In [80]:
mean_mag_tf= mean_mags[0].numpy()[:N+4]
for i in range(N):
    print(i+3, mean_mag_tf[i])
print('final mean mag ', mean_mag_tf[N-1]) # N-1 because we count from 0

3 18.545557
4 18.530783
5 18.514545
6 18.491264
7 18.357082
8 18.34712
9 18.324821
10 18.349573
11 18.287626
12 18.26964
13 18.292418
final mean mag  18.292418


In [79]:
batch[0]['input_LC_0'][0].numpy()[:N]

array([[ 1.4008764 ,  1.830699  ,  0.49304435, -0.72241086],
       [ 1.830699  ,  1.4313586 , -0.72241086,  0.2581554 ],
       [ 1.4313586 ,  1.8500041 ,  0.2581554 , -0.03686545],
       [ 1.8500041 ,  1.4696567 , -0.03686545, -0.07472663],
       [ 1.4696567 ,  1.7681835 , -0.07472663, -0.82287806],
       [ 1.7681835 ,  1.6156642 , -0.82287806,  0.72539264],
       [ 1.6156642 ,  1.2827781 ,  0.72539264, -0.13093238],
       [ 1.2827781 ,  1.8092461 , -0.13093238,  0.4258857 ],
       [ 1.8092461 ,  1.4851803 ,  0.4258857 , -0.9041752 ],
       [ 1.4851803 ,  1.576728  , -0.9041752 ,  0.40365925],
       [ 1.576728  ,  1.4341837 ,  0.40365925,  0.49392572]],
      dtype=float32)